Capstone Flask API - Andina

In [2]:
import flask
import pandas 
import requests
import gunicorn

Data in use: Chinook

In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/chinook.db")

In [4]:
data = pd.read_sql_query("""SELECT invoices.InvoiceDate, 
                                invoices.BillingCountry as Country, 
                                invoice_items.Quantity, 
                                invoice_items.UnitPrice,
                                tracks.AlbumId as Album, 
                                tracks.Name as tracks_name, 
                                genres.Name as genre 
                                FROM invoice_items 
                                LEFT JOIN invoices 
                                ON invoices.InvoiceId = invoice_items.InvoiceId 
                                LEFT JOIN tracks 
                                ON tracks.TrackId = invoice_items.TrackId 
                                LEFT JOIN genres 
                                ON tracks.GenreId = genres.GenreId 
                                """, 
                              conn, parse_dates = 'InvoiceDate')
data.head(20)


,InvoiceDate,Country,Quantity,UnitPrice,Album,tracks_name,genre
0,2009-01-01,Germany,1,0.99,2,Balls to the Wall,Rock
1,2009-01-01,Germany,1,0.99,3,Restless and Wild,Rock
2,2009-01-02,Norway,1,0.99,1,Put The Finger On You,Rock
3,2009-01-02,Norway,1,0.99,1,Inject The Venom,Rock
4,2009-01-02,Norway,1,0.99,1,Evil Walks,Rock
5,2009-01-02,Norway,1,0.99,1,Breaking The Rules,Rock
6,2009-01-03,Belgium,1,0.99,4,Dog Eat Dog,Rock
7,2009-01-03,Belgium,1,0.99,4,Overdose,Rock
8,2009-01-03,Belgium,1,0.99,5,Love In An Elevator,Rock
9,2009-01-03,Belgium,1,0.99,5,Janie's Got A Gun,Rock


In [143]:
# static country
data['year'] = data["InvoiceDate"].dt.year
country_genre = data.groupby(['year','Country','genre']).UnitPrice.sum().dropna().reset_index()
pd.DataFrame(country_genre)

,year,Country,genre,UnitPrice
0,2009,Australia,Metal,2.97
1,2009,Australia,Rock,8.91
2,2009,Austria,Metal,1.98
3,2009,Belgium,Metal,0.99
4,2009,Belgium,Rock,5.94
...,...,...,...,...
427,2013,USA,Rock,37.62
428,2013,United Kingdom,Jazz,2.97
429,2013,United Kingdom,Latin,15.84
430,2013,United Kingdom,Metal,5.94


In [ ]:
@app.route('/country')
def country():
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("""SELECT invoices.InvoiceDate, 
                                invoices.BillingCountry as Country, 
                                invoice_items.Quantity, 
                                invoice_items.UnitPrice,
                                tracks.AlbumId as Album, 
                                tracks.Name as tracks_name, 
                                genres.Name as genre 
                                FROM invoice_items 
                                LEFT JOIN invoices 
                                ON invoices.InvoiceId = invoice_items.InvoiceId 
                                LEFT JOIN tracks 
                                ON tracks.TrackId = invoice_items.TrackId 
                                LEFT JOIN genres 
                                ON tracks.GenreId = genres.GenreId 
                                """, 
                              conn, parse_dates = 'InvoiceDate')
    data['year'] = data["InvoiceDate"].dt.to_period('Y')
    country_genre = data.groupby(['year','Country','genre']).UnitPrice.sum().dropna().reset_index()


    return (pd.DataFrame(country_genre).to_json())

In [144]:
url3 = 'http://localhost:5000/country'
r = requests.get(url3)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,year,Country,genre,UnitPrice
0,2009,Australia,Metal,2.97
1,2009,Australia,Rock,8.91
2,2009,Austria,Metal,1.98
3,2009,Belgium,Metal,0.99
4,2009,Belgium,Rock,5.94


In [134]:
# static yearly sales 
data['year'] = data["InvoiceDate"].dt.year
yearly_sales = pd.pivot_table(data= data,
                        index= ['year','Country'],
                        values= 'UnitPrice',
                        aggfunc= 'sum' 
                      ).reset_index().groupby(['year','Country']).UnitPrice.sum()
sales = pd.DataFrame(yearly_sales).reset_index()

In [9]:
import flask
import pandas as pd
import requests
import gunicorn


from flask import Flask, request 
app = Flask(__name__) 

@app.route('/sales')
def sales():
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("""SELECT invoices.InvoiceDate, 
                                invoices.BillingCountry as Country, 
                                invoice_items.Quantity, 
                                invoice_items.UnitPrice,
                                tracks.AlbumId as Album, 
                                tracks.Name as tracks_name, 
                                genres.Name as genre 
                                FROM invoice_items 
                                LEFT JOIN invoices 
                                ON invoices.InvoiceId = invoice_items.InvoiceId 
                                LEFT JOIN tracks 
                                ON tracks.TrackId = invoice_items.TrackId 
                                LEFT JOIN genres 
                                ON tracks.GenreId = genres.GenreId 
                                """, 
                              conn, parse_dates = 'InvoiceDate')
    data['year'] = data["InvoiceDate"].dt.to_period('Y')
    yearly_sales = pd.pivot_table(data= data,
                        index= ['year','Country'],
                        values= 'UnitPrice',
                        aggfunc= 'sum' 
                      ).reset_index().groupby(['year','Country']).UnitPrice.sum()
    
    return (pd.DataFrame(yearly_sales).to_json())

if __name__ == '__main__':
    app.run(debug=False, port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


OSError: [Errno 48] Address already in use

In [135]:
url1 = 'http://localhost:5000/sales'
r = requests.get(url1)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,year,Country,UnitPrice
0,2009,Australia,11.88
1,2009,Austria,1.98
2,2009,Belgium,6.93
3,2009,Brazil,37.62
4,2009,Canada,57.42


In [137]:
# most genre dynamic
data['genre'] = data['genre'].astype('category')
most_genre = pd.pivot_table(data= data,
                        index= 'genre',
                        columns= 'Country',
                        values= 'Quantity',
                        aggfunc= sum 
                      )
genre_country = most_genre.reset_index().melt(id_vars='genre',value_name='Total_Qty').dropna()
genre_country
genre_country[genre_country['genre']== 'Jazz']

,genre,Country,Total_Qty
1,Alternative & Punk,Argentina,9.0
7,Easy Listening,Argentina,2.0
11,Jazz,Argentina,2.0
12,Latin,Argentina,8.0
13,Metal,Argentina,7.0
...,...,...,...
566,Pop,United Kingdom,2.0
567,R&B/Soul,United Kingdom,2.0
568,Reggae,United Kingdom,5.0
569,Rock,United Kingdom,37.0


In [92]:
import flask
import pandas as pd
import requests
import gunicorn


from flask import Flask, request 
app = Flask(__name__) 

# mendapatkan keseluruhan data dari <data_name>
@app.route('/data/get/<genre>', methods=['GET']) 
def get_data(genre): 
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("""SELECT invoices.InvoiceDate, 
                                invoices.BillingCountry as Country, 
                                invoice_items.Quantity, 
                                invoice_items.UnitPrice,
                                tracks.AlbumId as Album, 
                                tracks.Name as tracks_name, 
                                genres.Name as genre 
                                FROM invoice_items 
                                LEFT JOIN invoices 
                                ON invoices.InvoiceId = invoice_items.InvoiceId 
                                LEFT JOIN tracks 
                                ON tracks.TrackId = invoice_items.TrackId 
                                LEFT JOIN genres 
                                ON tracks.GenreId = genres.GenreId 
                                """, 
                              conn, parse_dates = 'InvoiceDate')
    
    data['genre'] = data['genre'].astype('category')
    most_genre = pd.pivot_table(data= data,
                        index= 'genre',
                        columns= 'Country',
                        values= 'Quantity',
                        aggfunc= sum 
                      )
    genre_country = most_genre.reset_index().melt(id_vars='genre').dropna()
    cond = genre_country['genre']==str(genre)
    
    return (genre_country[cond].to_json())

if __name__ == '__main__':
    app.run(debug=False, port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


OSError: [Errno 48] Address already in use

In [138]:
url2 = 'http://localhost:5000/data/get/Jazz'
r = requests.get(url2)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,genre,Country,Total_Qty
11,Jazz,Argentina,2.0
59,Jazz,Austria,2.0
131,Jazz,Canada,13.0
179,Jazz,Czech Republic,3.0
227,Jazz,Finland,2.0
